In [1]:
from  keras.wrappers.scikit_learn  import  KerasClassifier
from  sklearn.model_selection  import  GridSearchCV
import numpy as np
# "Sequential" models let us define a stack of neural network layers
from keras.models import Sequential

# import the core layers:
from keras.layers import Dense, Dropout, Activation, Flatten

# import some utilities to transform/preprocess our data:
from keras.utils import np_utils

# Keras will download MNIST digit dataset for us:
from keras.datasets import mnist


2023-03-13 21:14:01.700858: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-13 21:14:03.400451: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-13 21:14:03.400477: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-13 21:14:07.200641: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
# By default, the first 60k of MNIST has been defined as training and the rest as testing set:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

print(X_train.shape)
print(X_test.shape)

(60000, 28, 28)
(10000, 28, 28)


In [3]:
# Reshape each image pixels into a row of feature table with 28*28=784 features (each pixel is a feature):
X_train = X_train.reshape(X_train.shape[0], 784)
X_test = X_test.reshape(X_test.shape[0], 784)

# simply scale the features to the range of [0,1]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

#output label:
print(y_train.shape)
print(y_train[:10])

(60000,)
[5 0 4 1 9 2 1 3 1 4]


In [4]:
# OneHotEncoding for the output label:
y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

In [5]:
# Label after OneHotEncoding:
print (y_train.shape)
print(y_train[:10,:])

(60000, 10)
[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]


In [6]:
# Declare Sequential model to build our network:
def model_creator():
    # define:
    input_size = 784
    hidden_neurons = 100
    out_size = 10
    model = Sequential()

    # design the structure:
    # -----------------------------------------
    # second layer: hidden layer:
    model.add(Dense(hidden_neurons, input_dim = input_size))  # Nuerons
    model.add(Activation('sigmoid')) # Activation

    # -----------------------------------------
    # third layer: output layer:
    model.add(Dense(out_size, input_dim = hidden_neurons))  # Nuerons
    model.add(Activation('softmax')) # Activation

   # compile:
    model.compile(loss='categorical_crossentropy', optimizer='adam',  metrics=['accuracy'])

   # return:
    return model

In [7]:
# Fix random state for reproducibility
seed = 2
np.random.seed(seed)

# instantiate the model:
model = KerasClassifier(build_fn=model_creator,verbose=2)

# define grid search params
batch_size = [30,50,100]
epochs = [10,15,20]

# create a dictionary for grid parameter:
param_grid = dict(batch_size = batch_size, epochs = epochs)
print(param_grid,'\n')

# create the grid and define the metric for evaluating the model:
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=10, scoring='neg_log_loss')
grid.fit(X_train,y_train)

/tmp/ipykernel_22328/2653492917.py:6: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=model_creator,verbose=2)


{'batch_size': [30, 50, 100], 'epochs': [10, 15, 20]} 



2023-03-13 21:14:13.524844: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-13 21:14:13.525065: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-13 21:14:13.525135: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-03-13 21:14:13.525204: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-03-13 21:14:13.525249: W tensorflow/c

Epoch 1/10
1800/1800 - 2s - loss: 0.4308 - accuracy: 0.8906 - 2s/epoch - 1ms/step
Epoch 2/10
1800/1800 - 1s - loss: 0.2094 - accuracy: 0.9398 - 1s/epoch - 746us/step
Epoch 3/10
1800/1800 - 1s - loss: 0.1558 - accuracy: 0.9554 - 1s/epoch - 762us/step
Epoch 4/10
1800/1800 - 1s - loss: 0.1231 - accuracy: 0.9648 - 1s/epoch - 752us/step
Epoch 5/10
1800/1800 - 1s - loss: 0.1003 - accuracy: 0.9714 - 1s/epoch - 725us/step
Epoch 6/10
1800/1800 - 1s - loss: 0.0844 - accuracy: 0.9757 - 1s/epoch - 687us/step
Epoch 7/10
1800/1800 - 1s - loss: 0.0709 - accuracy: 0.9799 - 1s/epoch - 684us/step
Epoch 8/10
1800/1800 - 1s - loss: 0.0606 - accuracy: 0.9834 - 1s/epoch - 694us/step
Epoch 9/10
1800/1800 - 1s - loss: 0.0520 - accuracy: 0.9856 - 1s/epoch - 704us/step
Epoch 10/10
1800/1800 - 1s - loss: 0.0445 - accuracy: 0.9880 - 1s/epoch - 693us/step
188/188 [==============================] - 0s 504us/step
Epoch 1/10
1800/1800 - 1s - loss: 0.4267 - accuracy: 0.8920 - 1s/epoch - 826us/step
Epoch 2/10
1800/1800

GridSearchCV(cv=10,
             estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x7f7250941cf0>,
             param_grid={'batch_size': [30, 50, 100], 'epochs': [10, 15, 20]},
             scoring='neg_log_loss')

In [8]:
print('The best params are:', grid.best_params_)
score = grid.best_estimator_.score(X_test,y_test)
print(f'The accuracy is: ', {score})

The best params are: {'batch_size': 50, 'epochs': 20}
200/200 - 0s - loss: 0.0729 - accuracy: 0.9778 - 223ms/epoch - 1ms/step
The accuracy is:  {0.9778000116348267}
